<a href="https://colab.research.google.com/github/sw-hacker/programs_for_apo/blob/master/three_plus_person_pairing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First for the full experience turn on corgi mode (Tools->Preferences->Miscellaneous). This is essential to run the code. I cannot stress it enough. 


Next you must edit your comma separated value (csv) file. These commas can get tricky, so you must remove any cell where a person could enter a comma.

Now open the .csv file in a text editor and replace all quotation marks (") with blanks ().

You are all set to run the code. Upload the csv file in the tab to the side as the name "GKYB for code.csv". Alternatively you can change the name in the code (line 5)


You can change the weight on line 5. It exponentially increases reliance on participant questions. If you go too high, it overcomes the program choosing people with overall low matching first. 

In [1]:
import numpy as np
import csv

csv_file_name = "GKYB for code.csv"
weight = 3
group_size = 3                #Must be 3 (Can be made to do groups of 4 quicky)

#MAIN
#Read in the Raw CSV file, data is stored in csv_data as an array
with open(csv_file_name, 'r') as f:
  reader = csv.reader(f, skipinitialspace=True, delimiter=',')
  headers = next(reader)
  csv_data = list(reader)
  pad = len(max(csv_data, key=len))
  csv_data_array = np.array([i + [0]*(pad-len(i)) for i in csv_data]).astype(str)

#Fill in person_cross_scores_array with scores using two for loops
person_cross_scores_array = np.ones((csv_data_array.shape[0], csv_data_array.shape[0]))
for j in range(person_cross_scores_array.shape[0]):
  for i in range(j+1):
    if(i == j): 
      person_cross_scores_array[i][j] = -10000*weight*weight*weight
    else:
      for k in range(5):
        if csv_data_array[i][k+1] == csv_data_array[j][k+1]:
          person_cross_scores_array[i][j] *= weight
      if ((csv_data_array[i][0] in csv_data_array[j].tolist()) or
          (csv_data_array[j][0] in csv_data_array[i].tolist())):
        person_cross_scores_array[i][j] = -1000*weight*weight*weight
      person_cross_scores_array[j][i] = person_cross_scores_array[i][j]

#Fill pairing_array choosing people with lowest match scores first
pairing_array = np.empty((int(csv_data_array.shape[0]/group_size), group_size+2), dtype=object)
for n in range(pairing_array.shape[0]):
  #find lowest and store name
  lowest = np.array([10000,0])
  for m in range(person_cross_scores_array.shape[0]):
     if np.sum(person_cross_scores_array[m]) < lowest[0]:
       lowest[0] = np.sum(person_cross_scores_array[m])
       lowest[1] = m
  pairing_array[n][0] = csv_data_array[lowest[1]][0]

  #get best match
  best = np.array([0,0])
  for m in range(person_cross_scores_array.shape[0]):
    if  person_cross_scores_array[lowest[1]][m] > best[0]:
      best[0] = person_cross_scores_array[lowest[1]][m]
      best[1] = m
  pairing_array[n][1] = csv_data_array[best[1]][0]
  pairing_array[n][2] = best[0]

  #get best double match
  double_best = np.array([-50,0])
  for m in range(person_cross_scores_array.shape[0]):
    if (person_cross_scores_array[lowest[1]][m]+
        person_cross_scores_array[best[1]][m]) > double_best[0]:
      double_best[0] = (person_cross_scores_array[lowest[1]][m]+
                        person_cross_scores_array[best[1]][m])
      double_best[1] = m
  pairing_array[n][3] = csv_data_array[double_best[1]][0]
  pairing_array[n][4] = double_best[0]

  #get second best double match for groups of four
  

  #remove row and column of all three/four chosen
  delete_array = ([lowest[1],best[1],double_best[1]])
  delete_array.sort(reverse = True)

  for x in range(len(delete_array)):
    csv_data_array = np.delete(csv_data_array, delete_array[x], 0)
    person_cross_scores_array = np.delete(person_cross_scores_array, delete_array[x], 0)
    person_cross_scores_array = np.delete(person_cross_scores_array, delete_array[x], 1)
  

print("Pairs")
print(pairing_array)
print("Unpaired")
print(csv_data_array[:,0])

FileNotFoundError: ignored